In [3]:
import pandas as pd
import cv2 
import numpy as np
import os

In [4]:
hdf="/kaggle/input/human-faces/Humans"
new_data=[]
for file in os.listdir(hdf):
    data=os.path.join(hdf,file)
    data_cv=cv2.imread(data)
    data_cv_resize=cv2.resize(data_cv,(256,256))
    new_data.append(data_cv_resize)

In [16]:
new_img=np.array(new_data)

In [17]:
new_img.shape

(7219, 256, 256, 3)

# # GAN

In [9]:
from keras.layers import Dense,Input,Conv2D,MaxPooling2D,Flatten,Dense,Conv2DTranspose,Dropout,BatchNormalization,Conv2DTranspose
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [10]:
def build_gen():
    inputs=Input(shape=(256,256,3))
    G=Conv2DTranspose(256,(3,3),strides=(2,2),activation="relu",padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(inputs)
    G=BatchNormalization()(G)
    G=Dropout(0.5)(G)
    G=Conv2DTranspose(128,(3,3),strides=(2,2),activation="relu",padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(G)
    G=BatchNormalization()(G)
    G=Dropout(0.5)(G)
    G=Conv2DTranspose(64,(3,3),strides=(2,2),activation="relu",padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(G)
    G=BatchNormalization()(G)
    G=Dropout(0.5)(G)
    G=Conv2DTranspose(32,(3,3),strides=(2,2),activation="relu",padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(G)
    G=BatchNormalization()(G)
    G=Flatten()(G)
    outputs=Dense(1,activation="relu")(G)
    model=Model(inputs,outputs)
    return model




def build_discrim():
    inputs=Input(shape=(256,256,3))
    G=Conv2D(32,(3,3),activation="relu",padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(inputs)
    G=MaxPooling2D(pool_size=(2,2),strides=(2,2))(G)
    G=BatchNormalization()(G)
    G=Dropout(0.5)(G)
    G=Conv2D(64,(3,3),activation="relu",padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(G)
    G=MaxPooling2D(pool_size=(2,2),strides=(2,2))(G)
    G=BatchNormalization()(G)
    G=Dropout(0.5)(G)
    G=Conv2D(128,(3,3),activation="relu",padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(G)
    G=MaxPooling2D(pool_size=(2,2),strides=(2,2))(G)
    G=BatchNormalization()(G)
    G=Dropout(0.5)(G)
    G=Conv2D(256,(3,3),activation="relu",padding='same',kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(G)
    G=MaxPooling2D(pool_size=(2,2),strides=(2,2))(G)
    G=BatchNormalization()(G)
    G=Flatten()(G)
    outputs=Dense(1,activation="relu")(G)
    model=Model(inputs,outputs)
    return model

def build_train(generator,discrim,dataset):
    discrim.trainalbe=False
    discrim.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])
    noise=np.random.normal(32,256,256,3)
    generator=generator.predict(noise)
    fake_img=generator.predict(noise)
    real_img=next(dataset)
    fake_label=np.zeros((32,1))
    real_label=np.ones((32,1))
    discrim.trainable=True
    
    dicri_real=discrim.train_on_batch(real_img,real_label)
    dicri_fake=discrim.train_on_batch(fake_img,fake_label)
    combine=Model(generator.input,discrim(generator.output))
    combine.combine(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return combine

In [ ]:
    generator=build_gen()
    discrim=build_discrim()
    model=build_train(generator,discrim,new_df)
    for epoch in range(10):
        noise=np.random.normal(32,256,256,3)
        fake_img=generator.predict(noise)
        real_img=next(new_img)
        fake_label=np.zeros((32,1))
        real_label=np.ones((32,1))
    
        discrim.trainable=True
        dicri_real=discrim.train_on_batch(real_img,real_label)
        dicri_fake=discrim.train_on_batch(fake_img,fake_label)
        discrim.trainalbe=False

        model.train_on_batch(noise,real_img)

# # Deep learning

In [21]:
inputs=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation='relu',padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)
C=Conv2D(64,(3,3),activation='relu',padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)
C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
C=Conv2D(128,(3,3),activation='relu',padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)
C=Conv2D(256,(3,3),activation='relu',padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)
C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)

C=Conv2D(556,(3,3),activation='relu',padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)

C=Conv2DTranspose(256,(3,3),activation='relu',padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)
C=Conv2DTranspose(128,(3,3),strides=(2,2),activation='relu',padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)
C=Conv2DTranspose(64,(3,3),activation='relu',padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)
C=Conv2DTranspose(32,(3,3),strides=(2,2),activation='relu',padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)
outputs=Conv2D(3,(3,3),activation="softmax",padding='same')(C)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_38          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_39          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 128, 128, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 128, 128, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 128, 128, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 64, 64, 556)    │     1,281,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 64, 64, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 64, 64, 556)    │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_17             │ (None, 64, 64, 256)    │     1,281,280 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 3,345,503 (12.76 MB)

 Trainable params: 3,342,471 (12.75 MB)

 Non-trainable params: 3,032 (11.84 KB)

In [ ]:
model.fit(new_img,new_img,epochs=3)

Epoch 1/3
 25/226 ━━━━━━━━━━━━━━━━━━━━ 2:37:12 47s/step - accuracy: 0.3393 - loss: 624.4158

In [ ]:
prediction=model.predict(new_img)
reconstruction_error=np.mean(np.square(new_img-prediction))
threshold=np.mean(reconstruction_error)+2*np.std(reconstruction_error)
label=np.where(reconstruction_error>threshold,0,1)
accuracy=np.mean(label==0)*100
accuracy